In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent.resolve()))

# Uncomment to get more debugging printouts:
"""
import logging

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

In [ ]:
from trulens_eval.keys import *

In [ ]:
# LLama Index starter example from: https://gpt-index.readthedocs.io/en/latest/getting_started/starter_example.html

from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('llama_index/data').load_data()
index = GPTVectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()
# response = query_engine.query("What did the author do growing up?")
# print(response)

In [ ]:
from trulens_eval import TruLlama, Tru, Query, Feedback
from trulens_eval import tru_feedback

tru = Tru()
Tru().reset_database()

proc = Tru().start_dashboard(force=True, _dev=Path.cwd().parent.parent)
thread = Tru().start_evaluator(restart=True)

In [ ]:
import numpy as np

# Construct feedback functions.

hugs = tru_feedback.Huggingface()
openai = tru_feedback.OpenAI()

# Language match between question/answer.
f_lang_match = Feedback(hugs.language_match).on(
    text1=Query.RecordInput, text2=Query.RecordOutput
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on(
    prompt=Query.RecordInput, response=Query.RecordOutput
)

# Question/statement relevance between question and each context chunk.
f_qs_relevance = tru_feedback.Feedback(openai.qs_relevance).on(
    question=Query.RecordInput,
    statement=Query.Record.model.retriever.retrieve.rets[:].node.text
).aggregate(np.min)

feedbacks = [f_lang_match, f_qa_relevance, f_qs_relevance]

l = tru.Llama(engine=query_engine, feedbacks=feedbacks)

In [ ]:
list(l.instrumented())

In [ ]:
res, record = l.query_with_record("What did the author do growing up?")

In [ ]:
record.dict()